In [3]:
from mmcv import __version__ as mmcvversion
print(mmcvversion)
from mmdet import __version__ as mmdetversion
print(mmdetversion)
from torch import __version__ as torchversion
print(torchversion)

2.0.1
3.3.0
2.0.0+cu118


In [4]:
import os
os.chdir('/workspace/objectdet')

In [6]:
from mmengine.config import Config

cfg = Config.fromfile('mmdetection/configs/dino/dino-5scale_swin-l_8xb2-12e_coco.py')

In [7]:
cfg.model.bbox_head.num_classes = 1
cfg.data_root = "datasets/Box-1/"

metainfo = {
    'classes': ('Box', ),
    'palette': [
        (220, 20, 60),
    ]
}

cfg.train_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=True),
    batch_sampler=dict(type='AspectRatioBatchSampler'),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='train/_annotations.coco.json',
        data_prefix=dict(img='train/'),
        filter_cfg=dict(filter_empty_gt=True, min_size=32),
        pipeline=cfg.train_pipeline,
        backend_args=cfg.backend_args))
cfg.val_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='valid/_annotations.coco.json',
        data_prefix=dict(img='valid/'),
        test_mode=True,
        pipeline=cfg.test_pipeline,
        backend_args=cfg.backend_args))

cfg.val_evaluator = dict(
    type='CocoMetric',
    ann_file=cfg.data_root + 'valid/_annotations.coco.json',
    metric='bbox',
    format_only=False,
    backend_args=cfg.backend_args)


# inference on test dataset and
# format the output results for submission.
cfg.test_dataloader = dict(
    batch_size=1,
    num_workers=2,
    persistent_workers=True,
    drop_last=False,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file=cfg.data_root + 'test/_annotations.coco.json',
        data_prefix=dict(img='test/'),
        test_mode=True,
        pipeline=cfg.test_pipeline))
cfg.test_evaluator = dict(
    type='CocoMetric',
    metric='bbox',
    format_only=True,
    ann_file=cfg.data_root + 'test/_annotations.coco.json',
    outfile_prefix='./work_dirs/dino-5scale_swin-l_1xb2-12e_boxes/test')


param_scheduler = [
    dict(
        type='MultiStepLR',
        begin=0,
        end=12,
        by_epoch=True,
        milestones=[11],
        gamma=0.1)
]

cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=12, val_interval=1)


cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(
        type='AdamW',
        lr=0.0001/8,  # 0.0002 for DeformDETR
        weight_decay=0.0001/8),
    clip_grad=dict(max_norm=0.1, norm_type=2),
    paramwise_cfg=dict(custom_keys={'backbone': dict(lr_mult=0.1)})
)  # custom_keys contains sampling_offsets and reference_points in DeformDETR  # noqa

# cfg.auto_scale_lr = dict(enable=False, base_batch_size=1)
cfg.work_dir = './work_dirs/dino-5scale_swin-l_1xb2-12e_boxes/1'
# cfg.auto_scale_lr = dict(base_batch_size=16)

cfg.default_hooks = dict(
                checkpoint=dict(
                    type='CheckpointHook', 
                    interval=3,
                    max_keep_ckpts=2,  # only keep latest 2 checkpoints
                    save_best='auto'
                    ),
                sampler_seed=dict(type='DistSamplerSeedHook'),
                param_scheduler=dict(type='ParamSchedulerHook'),
                logger=dict(type='LoggerHook', interval=10),
                timer=dict(type='IterTimerHook'),
                visualization=dict(type='DetVisualizationHook'),
                )
# cfg.vis_backends = dict(type='LocalVisBackend')


In [8]:
from mmengine.runner import Runner

model = Runner.from_cfg(cfg)

model.train()

03/15 09:07:51 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1057011217
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencod

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window12_384_22k.pth" to /root/.cache/torch/hub/checkpoints/swin_large_patch4_window12_384_22k.pth


03/15 09:08:19 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
03/15 09:08:19 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.
03/15 09:08:19 - mmengine - INFO - Checkpoints will be saved to /workspace/objectdet/work_dirs/dino-5scale_swin-l_1xb2-12e_boxes/1.


/workspace/objectdet/.venv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


03/15 09:11:47 - mmengine - INFO - Epoch(train)  [1][ 10/460]  base_lr: 1.2500e-05 lr: 1.2500e-05  eta: 1 day, 7:48:07  time: 20.7782  data_time: 0.0210  memory: 10040  grad_norm: 131.4958  loss: 50.3786  loss_cls: 0.8726  loss_bbox: 1.7144  loss_iou: 1.6424  d0.loss_cls: 0.8701  d0.loss_bbox: 1.7817  d0.loss_iou: 1.6540  d1.loss_cls: 0.7946  d1.loss_bbox: 1.8919  d1.loss_iou: 1.5410  d2.loss_cls: 0.9547  d2.loss_bbox: 1.8896  d2.loss_iou: 1.5427  d3.loss_cls: 1.1328  d3.loss_bbox: 1.7243  d3.loss_iou: 1.6594  d4.loss_cls: 0.9864  d4.loss_bbox: 1.7224  d4.loss_iou: 1.6495  enc_loss_cls: 0.8914  enc_loss_bbox: 1.7179  enc_loss_iou: 1.6428  dn_loss_cls: 0.6822  dn_loss_bbox: 1.3571  dn_loss_iou: 1.3825  d0.dn_loss_cls: 0.7867  d0.dn_loss_bbox: 1.3566  d0.dn_loss_iou: 1.3829  d1.dn_loss_cls: 0.6035  d1.dn_loss_bbox: 1.3567  d1.dn_loss_iou: 1.3828  d2.dn_loss_cls: 0.5445  d2.dn_loss_bbox: 1.3568  d2.dn_loss_iou: 1.3828  d3.dn_loss_cls: 0.4747  d3.dn_loss_bbox: 1.3569  d3.dn_loss_iou: 1.382

KeyboardInterrupt: 